In [1]:
import glob
import os
import time
import pickle

from lsst.obs.lsst import LsstCamMapper as camMapper
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms

from mixcoatl.crosstalkTask import CrosstalkTask

## Crosstalk Task Debug

In [2]:
## Script command line arguments
sensor_id = 'R21_S11'
calib_dir= '/nfs/slac/g/ki/ki19/lsst/snyder18/LSST/Data/BOT/6813D_xtalk/calibration/'
output_dir = './'
main_dir = '/gpfs/slac/lsst/fs3/g/data/jobHarness/jh_archive-test/LCA-10134_Cryostat/LCA-10134_Cryostat-0001/6813D/BOT_acq/v0/47875/'
outfile = 'test_crosstalk_matrix.fits'

camera = camMapper._makeCamera()
lct = LsstCameraTransforms(camera)
gain_results = pickle.load(open(os.path.join(calib_dir, 
                                             'et_results.pkl'), 'rb'))
subdir_list = [x.path for x in os.scandir(main_dir) if os.path.isdir(x.path)]

bias_frame = os.path.join(calib_dir, '{0}_superbias.fits'.format(sensor_id))
dark_frame = os.path.join(calib_dir, '{0}_superdark.fits'.format(sensor_id))
gains = gain_results.get_amp_gains(sensor_id)

position_set = set()
for subdir in subdir_list:
    basename = os.path.basename(subdir)
    if "xtalk" not in basename:
        continue
    xpos, ypos = basename.split('_')[-4:-2]
    central_sensor, ccdX, ccdY = lct.focalMmToCcdPixel(float(ypos), float(xpos))
    if central_sensor == sensor_id:
        position_set.add((xpos, ypos))

## Only do one for now
for i, pos in enumerate(position_set):
    pos = list(position_set)[0]
    xpos, ypos = pos
    infiles = glob.glob(os.path.join(main_dir, '*{0}_{1}*'.format(xpos, ypos),
                                     '*{0}.fits'.format(sensor_id)))
    
    a = time.time()
    if i == 0:
        crosstalk_task = CrosstalkTask()
        crosstalk_task.config.outfile = outfile
        crosstalk_task.run(sensor_id, infiles[0], gains, bias_frame=bias_frame,
                           dark_frame=dark_frame)
    else:
        crosstalk_task = CrosstalkTask()
        crosstalk_task.run(sensor_id, infiles[0], gains, bias_frame=bias_frame,
                           dark_frame=dark_frame, crosstalk_matrix_file=outfile)
    print("Total Runtime:", time.time()-a)
        



Total Runtime: 14.972338438034058
Total Runtime: 11.100889444351196
Total Runtime: 11.636770486831665
Total Runtime: 11.99172854423523


In [80]:
12*4/60.

0.8